/content/drive/MyDrive/test.csv
/content/drive/MyDrive/rosstat.csv

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path=input()
df_bonus = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/rosstat.csv


In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

def full_tab(df):
  df = df.replace ( r'^\s\*$' , np.nan , regex= True )
  df=df.dropna()
  df=df.reset_index(drop=True)
  return df

def sum_data(t,data):
  summ=0
  for i in range(t):
      summ+=int(data[i])
  return summ

def T_finder(t,data_t,data):
  for tau in range(len(data)):
    if data_t[tau]>=data[t] or tau>=t:
      return tau

def pers_next(df_bonus):
  # fit model
  modelB = ARIMA(df_bonus['rBt'], order=(5,1,0))
  modelB_fit = modelB.fit()
  outputB = modelB_fit.forecast()
  #summary of fit model
  # print(modelB_fit.summary())


  # fit model
  modelDNM = ARIMA(df_bonus['rDNMt'], order=(5,1,0))
  modelDNM_fit = modelDNM.fit()
  outputDNM = modelDNM_fit.forecast()
  #summary of fit model
  # print(modelDNM_fit.summary())
  # print(outputB, outputDNM)
  return [outputB, outputDNM]

def get_next(n0,prevB, persB, prevDNM, persDNM):
  nextBt=(1+persB/100)*prevB
  nextDNMt=(1+persDNM/100)*prevDNM
  nt=n0+nextBt-nextDNMt
  return [nextBt, nextDNMt, nt]


def get_pred(df_bonus,years):
  df_bonus = full_tab(df_bonus)
  df_bonus = df_bonus.assign(DNM= lambda x: (x['D(t)']) + x['NM(t)'])

  print(df_bonus)

  Intb_sum=[sum_data(i+1, df_bonus['B(t)']) for i in range(len(df_bonus['B(t)']))]
  df_bonus = df_bonus.assign(IntB= Intb_sum)

  IntDNM_sum=[sum_data(i+1, df_bonus['DNM']) for i in range(len(df_bonus['DNM']))]
  df_bonus = df_bonus.assign(IntDNM= IntDNM_sum)

  rBt_t=[0]+[100*(df_bonus['IntB'][i] - df_bonus['IntB'][i-1])/df_bonus['IntB'][i] for i in range(1,len(df_bonus['IntB']))]
  df_bonus = df_bonus.assign(rBt= rBt_t)

  rDNMt_t=[0]+[100*(df_bonus['IntDNM'][i] - df_bonus['IntDNM'][i-1])/df_bonus['IntDNM'][i] for i in range(1,len(df_bonus['IntDNM']))]
  df_bonus = df_bonus.assign(rDNMt= rDNMt_t)

  tau=[0]+[i-T_finder(i,df_bonus['IntB'],df_bonus['IntDNM']) for i in range(1,len(df_bonus['IntDNM']))]
  df_bonus = df_bonus.assign(Qt= tau)

  for i in range(years):
    pers=pers_next(df_bonus)
    new_row_list=get_next(df_bonus['N(t)'][0], df_bonus['IntB'][len(df_bonus['IntB'])-1], pers[0], df_bonus['IntDNM'][len(df_bonus['IntDNM'])-1],pers[1])
    new_row_dict={'Year':df_bonus['Year'][len(df_bonus['Year'])-1]+1, 'N(t)':int(new_row_list[2]), 'IntB':int(new_row_list[0]), 'IntDNM':int(new_row_list[1]), 'rBt':float(pers[0]), 'rDNMt':float(pers[1])}
    df_bonus=df_bonus.append(new_row_dict, ignore_index=True)

  tau=[0]+[i-T_finder(i,df_bonus['IntB'],df_bonus['IntDNM']) for i in range(1,len(df_bonus['IntDNM']))]
  df_bonus = df_bonus.assign(Qt= tau)

  return df_bonus

In [ ]:
df_bonus=get_pred(df_bonus,10)
df_bonus.to_excel("output.xlsx")

      Year         N(t)      B(t)      D(t)     NM(t)       DNM       IntB  \
0   2002.0  145200000.0  398911.0  693450.0   87149.0  606301.0   398911.0   
1   2003.0  145000000.0  426736.0  708257.0   43884.0  664373.0   825647.0   
2   2004.0  144300000.0  428230.0  688508.0   41275.0  647233.0  1253877.0   
3   2005.0  143800000.0  420506.0  708173.0  107432.0  600741.0  1674383.0   
4   2006.0  143200000.0  435097.0  665458.0  132319.0  533139.0  2109480.0   
5   2007.0  142800000.0  489381.0  635034.0  239943.0  395091.0  2598861.0   
6   2008.0  142800000.0  519127.0  632425.0  242106.0  390319.0  3117988.0   
7   2009.0  142700000.0  524072.0  612952.0  247449.0  365503.0  3642060.0   
8   2010.0  142900000.0  525055.0  606782.0  158078.0  448704.0  4167115.0   
9   2011.0  142900000.0  526582.0  569024.0  319761.0  249263.0  4693697.0   
10  2012.0  143000000.0  546410.0  552700.0  294930.0  257770.0  5240107.0   
11  2013.0  143300000.0  538512.0  539304.0  295859.0  243445.0 

<ipython-input-77-02f7ae8899ef>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_bonus=df_bonus.append(new_row_dict, ignore_index=True)
<ipython-input-77-02f7ae8899ef>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_bonus=df_bonus.append(new_row_dict, ignore_index=True)
<ipython-input-77-02f7ae8899ef>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_bonus=df_bonus.append(new_row_dict, ignore_index=True)
<ipython-input-77-02f7ae8899ef>:72: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_bonus=df_bonus.append(new_row_dict, ignore_index=True)
<ipython-input-77-02f7ae8899ef>:72: FutureWarning: The frame.append method is deprec